In [48]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
from rake_nltk import Rake

In [2]:
base_url='https://www.goodreads.com'

In [5]:
r = requests.get('https://www.goodreads.com/list/show/3810.Best_Cozy_Mystery_Series?page=1')
soup = BeautifulSoup(r.text, 'html.parser')

In [6]:
link = soup.find('a', class_='bookTitle')
href = link['href']
print(href)

/book/show/16331.Murder_at_the_Vicarage


In [7]:

product_links = set()

for x in range(0,2):  
    r = requests.get(f'https://www.goodreads.com/list/show/3810.Best_Cozy_Mystery_Series?page={x}')
    soup = BeautifulSoup(r.text, 'html.parser')
    
    product_list = soup.find_all('a', class_='bookTitle', href=True)
    
    for link in product_list:
        product_links.add(base_url + link['href'])

product_links = list(product_links)


In [8]:
product_links

['https://www.goodreads.com/book/show/8145181-truly-madly',
 'https://www.goodreads.com/book/show/33445.Blueberry_Muffin_Murder',
 'https://www.goodreads.com/book/show/102303.Why_Didn_t_They_Ask_Evans_',
 'https://www.goodreads.com/book/show/960.Angels_Demons',
 'https://www.goodreads.com/book/show/586437.The_Tale_of_Hill_Top_Farm',
 'https://www.goodreads.com/book/show/6372680-aunt-dimity-s-death',
 'https://www.goodreads.com/book/show/512385.The_Trouble_With_Magic',
 'https://www.goodreads.com/book/show/52843028-the-mysterious-affair-at-styles',
 'https://www.goodreads.com/book/show/3663410-the-diva-runs-out-of-thyme',
 'https://www.goodreads.com/book/show/156510.The_Ghost_and_Mrs_McClure',
 'https://www.goodreads.com/book/show/36347162-chocolate-chip-cookie-murder',
 'https://www.goodreads.com/book/show/90187.Knit_One_Kill_Two',
 'https://www.goodreads.com/book/show/45020.Shakespeare_s_Landlord',
 'https://www.goodreads.com/book/show/2685034-one-bad-apple',
 'https://www.goodreads.c

In [10]:
test_link="https://www.goodreads.com/book/show/16331.Murder_at_the_Vicarage"

In [11]:
r=requests.get(test_link)
soup=BeautifulSoup(r.text,'lxml')

In [13]:
soup.find('h1').text

'Murder at the Vicarage'

In [14]:
Name=soup.find('h1').text
Name

'Murder at the Vicarage'

In [15]:
Contributor_Name= soup.find('span', class_='ContributorLink__name').text
Contributor_Name

'Agatha Christie'

In [16]:
rating_text = soup.find('span', class_='RatingStars RatingStars__medium')['aria-label']
rating = float(rating_text.split()[1])
print(rating)

4.05


In [17]:
rating_count_text = soup.find('span', {'data-testid': 'ratingsCount'}).text
rating_count = int(rating_count_text.replace(',', '').split()[0])
rating_count

187812

In [18]:
reviews_count_text = soup.find('span', class_='u-dot-before').text
reviews_count = int(reviews_count_text.replace(',', '').split()[0])
reviews_count

7351

In [19]:
Format=soup.find('p').text
Format

'288 pages, Hardcover'

In [28]:
product_data = []

for link in product_links:
    try:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'lxml')

        try:
            h1_tag = soup.find('h1')
            if h1_tag:
                Name = h1_tag.text.strip()
            else:
                Name = None
        except AttributeError:
            print("Error occurred while scraping 'h1' tag in link:", link)
            Name = None

        try:
            contributor_tag = soup.find('span', class_='ContributorLink__name')
            if contributor_tag:
                Contributor_Name = contributor_tag.text.strip()
            else:
                Contributor_Name = None
        except AttributeError:
            print("Error occurred while scraping 'ContributorLink__name' in link:", link)
            Contributor_Name = None

        rating_text = soup.find('span', class_='RatingStars RatingStars__medium')
        if rating_text:
            try:
                rating = float(rating_text['aria-label'].split()[1])
            except (IndexError, ValueError):
                print("Error occurred while scraping 'rating' in link:", link)
                rating = None
        else:
            print("Error occurred while scraping 'rating' in link:", link)
            rating = None

        try:
            rating_count_text = soup.find('span', {'data-testid': 'ratingsCount'}).text
            rating_count = int(rating_count_text.replace(',', '').split()[0])
        except (AttributeError, ValueError):
            print("Error occurred while scraping 'rating_count' in link:", link)
            rating_count = None

        try:
            reviews_count_text = soup.find('span', class_='u-dot-before').text
            reviews_count = int(reviews_count_text.replace(',', '').split()[0])
        except (AttributeError, ValueError):
            print("Error occurred while scraping 'reviews_count' in link:", link)
            reviews_count = None

        try:
            Format = soup.find('p').text.strip()
        except AttributeError:
            print("Error occurred while scraping 'Format' in link:", link)
            Format = None

        Book = {'Name': Name, 'Author': Contributor_Name, 'rating': rating, 'rating_count': rating_count,
                'reviews_count': reviews_count, 'Format': Format}
        print(Book)
        product_data.append(Book)
    except requests.exceptions.RequestException as e:
        print("Error occurred while making a request to the link:", link)
        print(e)


{'Name': 'Truly, Madly', 'Author': 'Heather Webber', 'rating': 3.92, 'rating_count': 6997, 'reviews_count': 1152, 'Format': '305 pages, Kindle Edition'}
{'Name': 'Blueberry Muffin Murder', 'Author': 'Joanne Fluke', 'rating': 3.89, 'rating_count': 24079, 'reviews_count': 1358, 'Format': '310 pages, Paperback'}
{'Name': "Why Didn't They Ask Evans?", 'Author': 'Agatha Christie', 'rating': 3.9, 'rating_count': 34010, 'reviews_count': 2596, 'Format': '288 pages, Mass Market Paperback'}
{'Name': 'Angels & Demons', 'Author': 'Dan       Brown', 'rating': 3.94, 'rating_count': 3184346, 'reviews_count': 33792, 'Format': '736 pages, Mass Market Paperback'}
{'Name': 'The Tale of Hill Top Farm', 'Author': 'Susan Wittig Albert', 'rating': 3.91, 'rating_count': 4017, 'reviews_count': 536, 'Format': '286 pages, Hardcover'}
{'Name': "Aunt Dimity's Death", 'Author': 'Nancy Atherton', 'rating': 3.96, 'rating_count': 13801, 'reviews_count': 1440, 'Format': '244 pages, Mass Market Paperback'}
{'Name': 'The

In [49]:
df=pd.DataFrame(product_data)

In [50]:
df

,Name,Author,rating,rating_count,reviews_count,Format
0,"Truly, Madly",Heather Webber,3.92,6997.0,1152.0,"305 pages, Kindle Edition"
1,Blueberry Muffin Murder,Joanne Fluke,3.89,24079.0,1358.0,"310 pages, Paperback"
2,Why Didn't They Ask Evans?,Agatha Christie,3.90,34010.0,2596.0,"288 pages, Mass Market Paperback"
3,Angels & Demons,Dan Brown,3.94,3184346.0,33792.0,"736 pages, Mass Market Paperback"
4,The Tale of Hill Top Farm,Susan Wittig Albert,3.91,4017.0,536.0,"286 pages, Hardcover"
...,...,...,...,...,...,...
95,Death of a Gossip,M.C. Beaton,3.66,18319.0,1560.0,"179 pages, Paperback"
96,Books Can Be Deceiving,Jenn McKinlay,3.84,10948.0,1430.0,"282 pages, Mass Market Paperback"
97,Thyme of Death,Susan Wittig Albert,3.84,8542.0,487.0,"320 pages, Mass Market Paperback"
98,Demons are a Ghoul's Best Friend,Victoria Laurie,4.06,6488.0,326.0,"282 pages, Mass Market Paperback"


In [66]:
test='https://www.goodreads.com/book/show/512385.The_Trouble_With_Magic?ac=1&from_search=true&qid=4pYf11Lurw&rank=1'
test1="https://www.goodreads.com/book/show/518518.The_Chocolate_Cat_Caper?ac=1&from_search=true&qid=DiZhQizuX4&rank=1"

In [67]:
r1=requests.get(test1)
soup1=BeautifulSoup(r1.text,'html.parser')

In [52]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'html.parser')

In [60]:
soup.find_all('dd')

[]

In [61]:
pages_format = soup.find('p', {'data-testid': 'pagesFormat'}).text
pages_format

'261 pages, Paperback'

In [68]:
pages_format = soup1.find('p', {'data-testid': 'pagesFormat'}).text
pages_format

'240 pages, Mass Market Paperback'